In [108]:

# Importing libraries
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import StratifiedKFold, KFold, train_test_split
from sklearn.metrics import cohen_kappa_score, confusion_matrix, make_scorer
from sklearn.base import clone
from sklearn.linear_model import ElasticNetCV, LassoCV, Lasso, LinearRegression
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor
from scipy.optimize import minimize
from scipy import stats
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
import warnings
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode
import plotly.figure_factory as ff
import optuna
import matplotlib.pyplot as plt
import seaborn as sns
import random

# Additional imports for specific functions

# Creating DataFrames for each coin
aave = pd.read_csv('../data/coin_Aave.csv')
binancecoin = pd.read_csv('../data/coin_BinanceCoin.csv')
bitcoin = pd.read_csv('../data/coin_Bitcoin.csv')
cardano = pd.read_csv('../data/coin_Cardano.csv')
chainlink = pd.read_csv('../data/coin_ChainLink.csv')
cosmos = pd.read_csv('../data/coin_Cosmos.csv')
cryptocomcoin = pd.read_csv('../data/coin_CryptocomCoin.csv')
dogecoin = pd.read_csv('../data/coin_Dogecoin.csv')
eos = pd.read_csv('../data/coin_EOS.csv')
ethereum = pd.read_csv('../data/coin_Ethereum.csv')
iota = pd.read_csv('../data/coin_Iota.csv')
litecoin = pd.read_csv('../data/coin_Litecoin.csv')
monero = pd.read_csv('../data/coin_Monero.csv')
nem = pd.read_csv('../data/coin_NEM.csv')
polkadot = pd.read_csv('../data/coin_Polkadot.csv')
solana = pd.read_csv('../data/coin_Solana.csv')
stellar = pd.read_csv('../data/coin_Stellar.csv')
tether = pd.read_csv('../data/coin_Tether.csv')
tron = pd.read_csv('../data/coin_Tron.csv')
uniswap = pd.read_csv('../data/coin_Uniswap.csv')
usdcoin = pd.read_csv('../data/coin_USDCoin.csv')
wrappedbitcoin = pd.read_csv('../data/coin_WrappedBitcoin.csv')
xrp = pd.read_csv('../data/coin_XRP.csv')
coins = [aave, binancecoin, bitcoin, cardano, chainlink, cosmos, cryptocomcoin, 
         dogecoin, eos, ethereum, iota, litecoin, monero, nem, polkadot, 
         solana, stellar, tether, tron, uniswap, usdcoin, wrappedbitcoin, xrp]
# Display TR for a specific coin, e.g., Monero
coin_info = []
for coin in coins:
    name = coin['Name'][0]  # Assuming 'Name' column exists (if not, adjust this)
    n = np.shape(coin)
    coin_info.append([name, n[0], n[1]])  # Append the coin name, rows, and columns

# Create a new DataFrame from the collected data
coin_info_df = pd.DataFrame(coin_info, columns=['Coin Name', 'Rows', 'Columns'])
coin_info_df = coin_info_df.set_index('Coin Name')
coin_info_df = coin_info_df.sort_values(by='Rows', ascending=False)
# Display the DataFrame
print(coin_info_df)
# Display TR for a specific coin, e.g., Monero
bitcoin.head()

                 Rows  Columns
Coin Name                     
Litecoin         2991       10
Bitcoin          2991       10
XRP              2893       10
Dogecoin         2760       10
Monero           2602       10
Stellar          2527       10
Tether           2318       10
NEM              2288       10
Ethereum         2160       10
IOTA             1484       10
EOS              1466       10
Binance Coin     1442       10
TRON             1392       10
Chainlink        1385       10
Cardano          1374       10
USD Coin         1002       10
Crypto.com Coin   935       10
Wrapped Bitcoin   888       10
Cosmos            845       10
Solana            452       10
Polkadot          320       10
Uniswap           292       10
Aave              275       10


,SNo,Name,Symbol,Date,High,Low,Open,Close,Volume,Marketcap
0,1,Bitcoin,BTC,2013-04-29 23:59:59,147.488007,134.000000,134.444000,144.539993,0.0,1.603769e+09
1,2,Bitcoin,BTC,2013-04-30 23:59:59,146.929993,134.050003,144.000000,139.000000,0.0,1.542813e+09
2,3,Bitcoin,BTC,2013-05-01 23:59:59,139.889999,107.720001,139.000000,116.989998,0.0,1.298955e+09
3,4,Bitcoin,BTC,2013-05-02 23:59:59,125.599998,92.281898,116.379997,105.209999,0.0,1.168517e+09
4,5,Bitcoin,BTC,2013-05-03 23:59:59,108.127998,79.099998,106.250000,97.750000,0.0,1.085995e+09


In [147]:
# Loop through each coin DataFrame and calculate TR
for df in coins:
    df[['TR', 'ATR','TR/OPEN','ATR/Open']] = pd.concat([
        df.High.sub(df.Low), 
        df.High.sub(df.Close.shift()).abs(), 
        df.Low.sub(df.Close.shift()).abs()], axis=1).max(1).to_frame('TR').assign(
            ATR=lambda x: x['TR'].ewm(span=14).mean(),
            TR_OPEN=lambda x: x['TR'] / df['Open'],
            ATR_OPEN=lambda x: x['TR_OPEN'] .ewm(span=14).mean()
        )
    
bitcoin.head(8)

,SNo,Name,Symbol,Date,High,Low,Open,Close,Volume,Marketcap,TR,ATR,ATR/Open,TR/OPEN
2990,2991,Bitcoin,BTC,2021-07-06 23:59:59,35038.536363,33599.916169,33723.509655,34235.193451,2.650126e+10,6.418992e+11,1438.620194,1438.620194,0.042659,0.042659
2989,2990,Bitcoin,BTC,2021-07-05 23:59:59,35284.344430,33213.661034,35284.344430,33746.002456,2.672155e+10,6.326962e+11,2070.683396,1777.225481,0.051245,0.058686
2988,2989,Bitcoin,BTC,2021-07-04 23:59:59,35937.567147,34396.477458,34665.564866,35287.779766,2.492431e+10,6.615748e+11,2191.564691,1935.504466,0.055819,0.063220
2987,2988,Bitcoin,BTC,2021-07-03 23:59:59,34909.259899,33402.696536,33854.421362,34668.548402,2.438396e+10,6.499397e+11,1885.083230,1920.079188,0.055777,0.055682
2986,2987,Bitcoin,BTC,2021-07-02 23:59:59,33939.588699,32770.680780,33549.600177,33897.048590,3.872897e+10,6.354508e+11,1897.867621,1914.284224,0.055984,0.056569
2985,2986,Bitcoin,BTC,2021-07-01 23:59:59,35035.982712,32883.781226,35035.982712,33572.117653,3.783896e+10,6.293393e+11,2152.201486,1969.334037,0.057244,0.061428
2984,2985,Bitcoin,BTC,2021-06-30 23:59:59,36074.759757,34086.151878,35908.388054,35040.837249,3.405904e+10,6.568525e+11,2502.642105,2081.713354,0.059867,0.069695
2983,2984,Bitcoin,BTC,2021-06-29 23:59:59,36542.111018,34252.484892,34475.559697,35867.777735,3.790146e+10,6.723334e+11,2289.626126,2122.378031,0.061148,0.066413


In [130]:

traces = []
buttons = []

for i, coin in enumerate(coins):
    trace = go.Candlestick(
        x=coin['Date'],
        open=coin['Open'],
        high=coin['High'],
        low=coin['Low'],
        close=coin['Close'],
        increasing_line_color='green',
        decreasing_line_color='red',
        name=coin['Name'][0]  # Coin name for the legend
    )
    
    # Add the trace to the traces list
    traces.append(trace)
    
    # Create a button for each coin to show the respective candlestick chart
    button = dict(
        args=[{'visible': [False] * i + [True] + [False] * (len(coins) - i - 1)}],
        label=coin['Name'][0],  # Coin name as the button label
        method='update'
    )
    buttons.append(button)

fig = go.Figure(data=traces)

fig.update_layout(
    updatemenus=[dict(
        type='buttons',
        x=1.05,  # Position buttons outside the plot area
        y=1,
        xanchor='left',
        yanchor='top',
        buttons=buttons
    )],
    title='Cryptocurrency Prices',
    xaxis_title='Date',
    yaxis_title='Price',
    margin=dict(l=20, r=40, t=40, b=20),
    height = 850,
    autosize=True,
    xaxis_rangeslider_visible=False
)

fig.show()



In [ ]:
coins_atr = []
for coin in coins:
    coins_atr.append({'Name': coin['Name'][0], 'ATR/Open': coin['ATR/Open'][-50:].mean()})
atr_df = pd.DataFrame(coins_atr).sort_values(by='ATR/Open', ascending=False,ignore_index=True).set_index('Name')
print(atr_df)


                 ATR/Open
Name                     
TRON             0.444102
Dogecoin         0.417628
Binance Coin     0.370956
Monero           0.321799
Chainlink        0.234116
IOTA             0.207722
Cardano          0.202148
Aave             0.188097
Cosmos           0.174252
XRP              0.160345
Ethereum         0.159572
EOS              0.158710
Uniswap          0.157879
Stellar          0.156660
NEM              0.134902
Crypto.com Coin  0.132054
Solana           0.125478
Polkadot         0.123504
Litecoin         0.108860
Bitcoin          0.080019
Wrapped Bitcoin  0.045551
USD Coin         0.023529
Tether           0.006243


In [ ]:
for coin in coins:
     coin.sort_values(by='Date', ascending=False, inplace=True)

TypeError: 'DataFrame' object is not callable